# Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import re
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("max_colwidth", None)
from tqdm import tqdm
import clip

/workspace/mtpenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load images metadata to a df

In [2]:
image_dir = "/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final"

image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp'}

image_data = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            image_path = os.path.join(root, file)
            image_data.append({'image_name': file, 'image_path': image_path})

images_metadata_df = pd.DataFrame(image_data)
print(images_metadata_df.shape)
images_metadata_df.head()

(302123, 2)


,image_name,image_path
0,000146954.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000146954.jpg
1,000321963.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000321963.jpg
2,000178662.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000178662.jpg
3,000289623.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000289623.jpg
4,000157681.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000157681.jpg


# Load captions to a df

In [3]:
ccneg_preprocessed_data = torch.load("/workspace/Dataset/conclip/ccneg_preprocessed.pt")

print(f"Keys: {list(ccneg_preprocessed_data.keys())}")

Keys: ['kept', 'dropped', 'image_paths', 'annotations', 'num_ops']


In [4]:
annotations = ccneg_preprocessed_data['annotations']

rows = []

for i in range(len(annotations)):
    ann = annotations[i]

    row = {
        'image_number': ann.get('image_number'),
        'file_extension':  ann.get('file_extension'),
        'true_caption': ann.get('caption'),
        'labels': ann.get('labels'),
        'negated_caption': ann.get('sop_data', {}).get('negative-prompt'),
        'url': ann.get('json', {}).get('url'),
        'subject': ann.get('sop_data', {}).get('sop_decomposition', {}).get('subject'),
        'object_predicate_pairs': ann.get('sop_data', {}).get('sop_decomposition', {}).get('object-predicate-pairs', {}),
        'predicate': ann.get('sop_data', {}).get('sop_decomposition', {}).get('predicate'),
        'negate_word_present': ann.get('negate_word_present'),
        'num_ops': ann.get('num_ops')
    }

    rows.append(row)

annotiation_df = pd.DataFrame(rows)
print(annotiation_df.shape)
annotiation_df.head(1)

(228246, 11)


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1


In [15]:
annotiation_df['image_name'] = annotiation_df['image_number'] + '.' + annotiation_df['file_extension']

data_df = pd.merge(annotiation_df, images_metadata_df, on='image_name', how='inner')
print(data_df.shape)
data_df.head(10)

(228246, 13)


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1,000000003.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000003.jpg
1,000000005,jpg,a woman walks her dog on the beach .,"water,beach,sea,shore,ocean,canidae,dog,sky,wave,coast,mudflat,dog walking,human,sand,photography",a woman walks on the beach without her dog,https://media.gettyimages.com/photos/woman-walks-her-dog-on-the-beach-on-october-21-2014-in-saltcoats-picture-id457587968,woman,"[{'object': 'dog', 'predicate': 'walks her on the beach'}]",None,True,1,000000005.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000005.jpg
2,000000009,jpg,close up portrait of a smiling middle aged woman sitting against white wall,"hair,sitting,facial expression,nose,arm,cheek,smile,chin,lip,hand,close-up,brown hair,finger,wool,fur",close up portrait of a smiling middle aged woman without white wall,http://l7.alamy.com/zooms/29164f933d7340be90af1ab4c91f3644/close-up-portrait-of-a-smiling-middle-aged-woman-sitting-against-white-hxeygn.jpg,portrait,"[{'object': 'woman', 'predicate': 'of a'}, {'object': 'wall', 'predicate': 'against white'}, {'object': 'age', 'predicate': 'middle'}, {'object': 'smile', 'predicate': 'smiling'}]",None,True,4,000000009.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000009.jpg
3,000000018,jpg,man sitting on floor beside a pool using laptop,"sitting,tablet computer,table,technology,grass,drinking,swimming pool,leisure,furniture,gadget,electronic device,vacation,laptop,stock photography,computer",man sitting on floor without a pool using laptop,https://media.gettyimages.com/photos/man-sitting-on-floor-beside-a-pool-using-laptop-picture-id588490995?s=612x612,man,"[{'object': 'floor', 'predicate': 'on'}, {'object': 'pool', 'predicate': 'beside a'}, {'object': 'laptop', 'predicate': 'using'}]",None,True,3,000000018.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000018.jpg
4,000000007,jpg,a beautiful day with some buildings and plants .,"residential area,property,home,building,house,real estate,neighbourhood,town,architecture,estate,suburb,driveway,tree,road surface,road",a beautiful day without plants,https://d1tq208oegmb9e.cloudfront.net/site_photos_image/dbx%3A/urban+project/orange+county/fullerton/chapman+villas/Photos/3.jpg,day,"[{'object': 'buildings', 'predicate': 'with some'}, {'object': 'plants', 'predicate': 'with some'}]",None,True,2,000000007.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000007.jpg
5,000000000,jpg,christmas tree on a black background .,"christmas tree,christmas decoration,font,text,graphic design,illustration,interior design,tree,christmas eve,ornament,fir,plant,pine,pine family,graphics","christmas tree, not on a black background",https://thumb1.shutterstock.com/display_pic_with_logo/261388/223876810/stock-vector-christmas-tree-on-a-black-background-vector-223876810.jpg,tree,"[{'object': 'background', 'predicate': 'on a'}]",None,True,1,000000000.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000000.jpg
6,000000011,jpg,festive banner with flags and an inscription .,"logo,flag,illustration,red,text,font,emblem,graphic design,graphics,label,symbol,art","festive banner with an inscription, but not with flags",https://thumb1.shutterstock.com/display_pic_with_logo/161878175/475029928/stock-vector-vector-festive-banner-with-flags-of-the-vietnam-and-an-inscription-socialist-republi

# Split this dataset into different negation types

In [16]:
def clean_caption(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)  # Remove punctuation
        text = text.strip()
        return text
    return text

data_df['true_caption_clean'] = data_df['true_caption'].apply(clean_caption)
data_df['negated_caption_clean'] = data_df['negated_caption'].apply(clean_caption)

In [17]:
syntactic_negators = {"no", "not", "never", "neither", "nobody", "nothing"}

def is_syntactic_negation(caption):
    tokens = caption.lower().split()
    return any(word in syntactic_negators for word in tokens)

def is_morphological_negation(caption):
    # Basic prefix patterns for morphological negation
    prefixes = ("un", "dis", "in", "im", "ir", "il", "non", "mis")
    words = caption.lower().split()
    return any(word.startswith(prefix) for word in words for prefix in prefixes)

lexical_negators = {"without", "lack", "absent", "avoid", "missing"}

def is_lexical_negation(caption):
    tokens = caption.lower().split()
    return any(word in lexical_negators for word in tokens)

def classify_negation(caption):
    if is_syntactic_negation(caption):
        return "syntactic"
    elif is_lexical_negation(caption):
        return "lexical/semantic"
    elif is_morphological_negation(caption):
        return "morphological"
    else:
        return "unknown/pragmatic"
    
data_df["negation_bucket"] = data_df["negated_caption_clean"].apply(classify_negation)

data_df.head()

,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1,000000003.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000003.jpg,actor attends the season premiere,actor not attending the season premiere,syntactic
1,000000005,jpg,a woman walks her dog on the beach .,"water,beach,sea,shore,ocean,canidae,dog,sky,wave,coast,mudflat,dog walking,human,sand,photography",a woman walks on the beach without her dog,https://media.gettyimages.com/photos/woman-walks-her-dog-on-the-beach-on-october-21-2014-in-saltcoats-picture-id457587968,woman,"[{'object': 'dog', 'predicate': 'walks her on the beach'}]",None,True,1,000000005.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000005.jpg,a woman walks her dog on the beach,a woman walks on the beach without her dog,lexical/semantic
2,000000009,jpg,close up portrait of a smiling middle aged woman sitting against white wall,"hair,sitting,facial expression,nose,arm,cheek,smile,chin,lip,hand,close-up,brown hair,finger,wool,fur",close up portrait of a smiling middle aged woman without white wall,http://l7.alamy.com/zooms/29164f933d7340be90af1ab4c91f3644/close-up-portrait-of-a-smiling-middle-aged-woman-sitting-against-white-hxeygn.jpg,portrait,"[{'object': 'woman', 'predicate': 'of a'}, {'object': 'wall', 'predicate': 'against white'}, {'object': 'age', 'predicate': 'middle'}, {'object': 'smile', 'predicate': 'smiling'}]",None,True,4,000000009.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000009.jpg,close up portrait of a smiling middle aged woman sitting against white wall,close up portrait of a smiling middle aged woman without white wall,lexical/semantic
3,000000018,jpg,man sitting on floor beside a pool using laptop,"sitting,tablet computer,table,technology,grass,drinking,swimming pool,leisure,furniture,gadget,electronic device,vacation,laptop,stock photography,computer",man sitting on floor without a pool using laptop,https://media.gettyimages.com/photos/man-sitting-on-floor-beside-a-pool-using-laptop-picture-id588490995?s=612x612,man,"[{'object': 'floor', 'predicate': 'on'}, {'object': 'pool', 'predicate': 'beside a'}, {'object': 'laptop', 'predicate': 'using'}]",None,True,3,000000018.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000018.jpg,man sitting on floor beside a pool using laptop,man sitting on floor without a pool using laptop,lexical/semantic
4,000000007,jpg,a beautiful day with some buildings and plants .,"residential area,property,home,building,house,real estate,neighbourhood,town,architecture,estate,suburb,driveway,tree,road surface,road",a beautiful day without plants,https://d1tq208oegmb9e.cloudfront.net/site_photos_image/dbx%3A/urban+project/orange+county/fullerton/chapman+villas/Photos/3.jpg,day,"[{'object': 'buildings', 'predicate': 'with some'}, {'object': 'plants', 'predicate': 'with some'}]",None,True,2,000000007.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000007.jpg,a beautiful day with some buildings and plants,a beautiful day without plants,lexical/semantic


In [18]:
buckets = data_df["negation_bucket"].unique()
print(f"Buckets: {buckets}")

bucket_datasets = {}

for bucket in buckets:
    bucket_datasets[bucket] = data_df[data_df["negation_bucket"] == bucket]

syntactic_df = bucket_datasets.get("syntactic")
print(f"Syntactic DF: {syntactic_df.shape}")

lexical_df = bucket_datasets.get("lexical/semantic")
print(f"Lexical DF: {lexical_df.shape}")


Buckets: ['syntactic' 'lexical/semantic']
Syntactic DF: (150735, 16)
Lexical DF: (77511, 16)


In [19]:
def evaluate_negation_sensitivity_clip(df, model, preprocess, device="cuda"):
    """
    Evaluate negation sensitivity score using CLIP on a dataframe.
    """
    similarity_true_list = []
    similarity_neg_list = []
    comparison_list = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating with CLIP"):
        image_path = row["image_path"]
        true_caption = row["true_caption"]
        negated_caption = row["negated_caption"]

        # Load and preprocess image
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

        # Tokenize captions
        text_tokens = clip.tokenize([true_caption, negated_caption]).to(device)

        # Compute features
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text_tokens)

            # Normalize
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            # Compute similarities
            similarities = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity_true = similarities[0][0].item()
            similarity_neg = similarities[0][1].item()

        # Store results
        similarity_true_list.append(similarity_true)
        similarity_neg_list.append(similarity_neg)
        comparison_list.append(int(similarity_true > similarity_neg))

    # Create results DataFrame
    results_df = df.copy()
    results_df["similarity_true"] = similarity_true_list
    results_df["similarity_neg"] = similarity_neg_list
    results_df["true_greater_than_negated"] = comparison_list

    negation_sensitivity_score = sum(comparison_list) / len(comparison_list)
    return results_df, negation_sensitivity_score

In [21]:
def load_checkpoint(model_name: str, checkpoint_path: str, device: str = "cuda"):
    """
    Load CLIP model with a custom checkpoint.
    """
    model, preprocess = clip.load(model_name, device=device)
    ckpt = torch.load(checkpoint_path, weights_only=False)
    model = model.float()
    model.load_state_dict(ckpt["model"])
    model = model.to(device)
    return model.eval(), preprocess

def load_negclip_checkpoint(model_name: str, checkpoint_path: str, device: str = "cuda"):
    model, preprocess = clip.load(model_name, device=device)

    # Load checkpoint
    ckpt = torch.load(checkpoint_path, map_location=device, weights_only=False)
    state_dict = ckpt["state_dict"]  # extract the actual model weights

    # Clean up keys (remove any "model." or "module." prefixes)
    cleaned_state_dict = {}
    for k, v in state_dict.items():
        new_key = k
        if k.startswith("model."):
            new_key = k[len("model."):]
        elif k.startswith("module."):
            new_key = k[len("module."):]
        cleaned_state_dict[new_key] = v

    # Load weights
    model.load_state_dict(cleaned_state_dict, strict=False)  # use strict=False to skip harmless mismatches
    model = model.to(device)
    return model.eval(), preprocess


# Load NEG-CLIP ICLR 2023

In [23]:
checkpoint_path = "/workspace/Models/negclip/negclip_iclr_2023.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "ViT-B/32"

model, preprocess = load_negclip_checkpoint(model_name, checkpoint_path, device)

In [24]:
results_df, score = evaluate_negation_sensitivity_clip(syntactic_df, model, preprocess, device)
print("Negation Sensitivity Score:", score)
results_df.head()

Evaluating with CLIP: 100%|██████████| 150735/150735 [1:40:07<00:00, 25.09it/s]  


Negation Sensitivity Score: 0.5937638902710054


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1,000000003.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000003.jpg,actor attends the season premiere,actor not attending the season premiere,syntactic,0.798340,0.201782,1
5,000000000,jpg,christmas tree on a black background .,"christmas tree,christmas decoration,font,text,graphic design,illustration,interior design,tree,christmas eve,ornament,fir,plant,pine,pine family,graphics","christmas tree, not on a black background",https://thumb1.shutterstock.com/display_pic_with_logo/261388/223876810/stock-vector-christmas-tree-on-a-black-background-vector-223876810.jpg,tree,"[{'object': 'background', 'predicate': 'on a'}]",None,True,1,000000000.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000000.jpg,christmas tree on a black background,christmas tree not on a black background,syntactic,0.352295,0.647949,0
6,000000011,jpg,festive banner with flags and an inscription .,"logo,flag,illustration,red,text,font,emblem,graphic design,graphics,label,symbol,art","festive banner with an inscription, but not with flags",https://thumb1.shutterstock.com/display_pic_with_logo/161878175/475029928/stock-vector-vector-festive-banner-with-flags-of-the-vietnam-and-an-inscription-socialist-republic-of-vietnam-475029928.jpg,banner,"[{'object': 'flags', 'predicate': 'with'}, {'object': 'inscription', 'predicate': 'with'}]",None,True,2,000000011.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000011.jpg,festive banner with flags and an inscription,festive banner with an inscription but not with flags,syntactic,0.484375,0.515625,0
9,000000024,jpg,actor attends the opening night after party,"premiere,carpet,red carpet,bow tie,hat,singer,fashion accessory,performance,tuxedo,formal wear,suit,musician",actor not attending the opening night after party,https://media.gettyimages.com/photos/actor-henry-kelemen-attends-the-our-new-girl-opening-night-after-at-picture-id450417500?s=612x612,actor,"[{'object': 'opening night after party', 'predicate': 'attends the'}]",None,True,1,000000024.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000024.jpg,actor attends the opening night after party,actor not attending the opening night after party,syntactic,0.692871,0.307373,1
13,000000029,jpg,a set of theatrical masks with different emotions,"white,face,mask,black-and-white,head,font,masque,headgear,costume,illustration,monochrome photography,pattern,smile,comedy","a set of theatrical masks, not with different emotions",https://thumb1.shutterstock.com/display_pic_with_logo/4089682/680796190/stock-vector-a-set-of-six-theatrical-masks-with-different-emotions-680796190.jpg,set,"[{'object': 'masks', 'predicate': 'of'}, {'object': 'emotions', 'predicate': 'with different'}]",None,True,2,000000029.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000029.jpg,a set of theatrical masks with different emotions,a set of theatrical masks not with different emotions,syntactic,0.577637,0.422607,1


In [25]:
results_df, score = evaluate_negation_sensitivity_clip(lexical_df, model, preprocess, device)
print(f"\nNegation Sensitivity Score: {score:.4f}")
results_df.head()


Evaluating with CLIP: 100%|██████████| 77511/77511 [50:38<00:00, 25.51it/s]  



Negation Sensitivity Score: 0.5953


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
1,000000005,jpg,a woman walks her dog on the beach .,"water,beach,sea,shore,ocean,canidae,dog,sky,wave,coast,mudflat,dog walking,human,sand,photography",a woman walks on the beach without her dog,https://media.gettyimages.com/photos/woman-walks-her-dog-on-the-beach-on-october-21-2014-in-saltcoats-picture-id457587968,woman,"[{'object': 'dog', 'predicate': 'walks her on the beach'}]",None,True,1,000000005.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000005.jpg,a woman walks her dog on the beach,a woman walks on the beach without her dog,lexical/semantic,0.269043,0.730957,0
2,000000009,jpg,close up portrait of a smiling middle aged woman sitting against white wall,"hair,sitting,facial expression,nose,arm,cheek,smile,chin,lip,hand,close-up,brown hair,finger,wool,fur",close up portrait of a smiling middle aged woman without white wall,http://l7.alamy.com/zooms/29164f933d7340be90af1ab4c91f3644/close-up-portrait-of-a-smiling-middle-aged-woman-sitting-against-white-hxeygn.jpg,portrait,"[{'object': 'woman', 'predicate': 'of a'}, {'object': 'wall', 'predicate': 'against white'}, {'object': 'age', 'predicate': 'middle'}, {'object': 'smile', 'predicate': 'smiling'}]",None,True,4,000000009.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000009.jpg,close up portrait of a smiling middle aged woman sitting against white wall,close up portrait of a smiling middle aged woman without white wall,lexical/semantic,0.751953,0.248047,1
3,000000018,jpg,man sitting on floor beside a pool using laptop,"sitting,tablet computer,table,technology,grass,drinking,swimming pool,leisure,furniture,gadget,electronic device,vacation,laptop,stock photography,computer",man sitting on floor without a pool using laptop,https://media.gettyimages.com/photos/man-sitting-on-floor-beside-a-pool-using-laptop-picture-id588490995?s=612x612,man,"[{'object': 'floor', 'predicate': 'on'}, {'object': 'pool', 'predicate': 'beside a'}, {'object': 'laptop', 'predicate': 'using'}]",None,True,3,000000018.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000018.jpg,man sitting on floor beside a pool using laptop,man sitting on floor without a pool using laptop,lexical/semantic,0.880859,0.119202,1
4,000000007,jpg,a beautiful day with some buildings and plants .,"residential area,property,home,building,house,real estate,neighbourhood,town,architecture,estate,suburb,driveway,tree,road surface,road",a beautiful day without plants,https://d1tq208oegmb9e.cloudfront.net/site_photos_image/dbx%3A/urban+project/orange+county/fullerton/chapman+villas/Photos/3.jpg,day,"[{'object': 'buildings', 'predicate': 'with some'}, {'object': 'plants', 'predicate': 'with some'}]",None,True,2,000000007.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000007.jpg,a beautiful day with some buildings and plants,a beautiful day without plants,lexical/semantic,0.822266,0.177856,1
7,000000001,jpg,item : drawing of a figure surrounded by person,"drawing,modern art,line,visual arts,art,sketch,artwork,photographic paper,painting,illustration,black-and-white",item : drawing without a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd48780900641ded7ab53d74fe86fe--figure-painting-figure-drawing.jpg,drawing,"[{'object': 'figure', 'predicate': 'of a'}, {'object': 'person', 'predicate': 'surrounded by'}]",None,True,2,000000001.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000001.jpg,item drawing of a figure surrounded by person,item drawing without a figure surrounded by person,lexical/semantic,0.106689,0.893066,0


# Load Clip CC12M Negfull ViT-B/32 LR-1e-8

In [28]:
checkpoint_path = "/workspace/Models/negclip/clip_cc12m_negfull_vit_b_32_lr1e-8_clw_0_99_mlw_0_01.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "ViT-B/32"

model, preprocess = load_negclip_checkpoint(model_name, checkpoint_path, device)

In [29]:
results_df, score = evaluate_negation_sensitivity_clip(syntactic_df, model, preprocess, device)
print("Negation Sensitivity Score:", score)
results_df.head()

Evaluating with CLIP: 100%|██████████| 150735/150735 [1:42:33<00:00, 24.49it/s]  


Negation Sensitivity Score: 0.8038544465452615


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1,000000003.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000003.jpg,actor attends the season premiere,actor not attending the season premiere,syntactic,0.945801,0.054199,1
5,000000000,jpg,christmas tree on a black background .,"christmas tree,christmas decoration,font,text,graphic design,illustration,interior design,tree,christmas eve,ornament,fir,plant,pine,pine family,graphics","christmas tree, not on a black background",https://thumb1.shutterstock.com/display_pic_with_logo/261388/223876810/stock-vector-christmas-tree-on-a-black-background-vector-223876810.jpg,tree,"[{'object': 'background', 'predicate': 'on a'}]",None,True,1,000000000.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000000.jpg,christmas tree on a black background,christmas tree not on a black background,syntactic,0.833496,0.166626,1
6,000000011,jpg,festive banner with flags and an inscription .,"logo,flag,illustration,red,text,font,emblem,graphic design,graphics,label,symbol,art","festive banner with an inscription, but not with flags",https://thumb1.shutterstock.com/display_pic_with_logo/161878175/475029928/stock-vector-vector-festive-banner-with-flags-of-the-vietnam-and-an-inscription-socialist-republic-of-vietnam-475029928.jpg,banner,"[{'object': 'flags', 'predicate': 'with'}, {'object': 'inscription', 'predicate': 'with'}]",None,True,2,000000011.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000011.jpg,festive banner with flags and an inscription,festive banner with an inscription but not with flags,syntactic,0.584961,0.414795,1
9,000000024,jpg,actor attends the opening night after party,"premiere,carpet,red carpet,bow tie,hat,singer,fashion accessory,performance,tuxedo,formal wear,suit,musician",actor not attending the opening night after party,https://media.gettyimages.com/photos/actor-henry-kelemen-attends-the-our-new-girl-opening-night-after-at-picture-id450417500?s=612x612,actor,"[{'object': 'opening night after party', 'predicate': 'attends the'}]",None,True,1,000000024.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000024.jpg,actor attends the opening night after party,actor not attending the opening night after party,syntactic,0.777344,0.222656,1
13,000000029,jpg,a set of theatrical masks with different emotions,"white,face,mask,black-and-white,head,font,masque,headgear,costume,illustration,monochrome photography,pattern,smile,comedy","a set of theatrical masks, not with different emotions",https://thumb1.shutterstock.com/display_pic_with_logo/4089682/680796190/stock-vector-a-set-of-six-theatrical-masks-with-different-emotions-680796190.jpg,set,"[{'object': 'masks', 'predicate': 'of'}, {'object': 'emotions', 'predicate': 'with different'}]",None,True,2,000000029.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000029.jpg,a set of theatrical masks with different emotions,a set of theatrical masks not with different emotions,syntactic,0.476562,0.523438,0


In [30]:
results_df, score = evaluate_negation_sensitivity_clip(lexical_df, model, preprocess, device)
print(f"\nNegation Sensitivity Score: {score:.4f}")
results_df.head()

Evaluating with CLIP: 100%|██████████| 77511/77511 [51:31<00:00, 25.07it/s]  



Negation Sensitivity Score: 0.7675


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
1,000000005,jpg,a woman walks her dog on the beach .,"water,beach,sea,shore,ocean,canidae,dog,sky,wave,coast,mudflat,dog walking,human,sand,photography",a woman walks on the beach without her dog,https://media.gettyimages.com/photos/woman-walks-her-dog-on-the-beach-on-october-21-2014-in-saltcoats-picture-id457587968,woman,"[{'object': 'dog', 'predicate': 'walks her on the beach'}]",None,True,1,000000005.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000005.jpg,a woman walks her dog on the beach,a woman walks on the beach without her dog,lexical/semantic,0.792969,0.206909,1
2,000000009,jpg,close up portrait of a smiling middle aged woman sitting against white wall,"hair,sitting,facial expression,nose,arm,cheek,smile,chin,lip,hand,close-up,brown hair,finger,wool,fur",close up portrait of a smiling middle aged woman without white wall,http://l7.alamy.com/zooms/29164f933d7340be90af1ab4c91f3644/close-up-portrait-of-a-smiling-middle-aged-woman-sitting-against-white-hxeygn.jpg,portrait,"[{'object': 'woman', 'predicate': 'of a'}, {'object': 'wall', 'predicate': 'against white'}, {'object': 'age', 'predicate': 'middle'}, {'object': 'smile', 'predicate': 'smiling'}]",None,True,4,000000009.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000009.jpg,close up portrait of a smiling middle aged woman sitting against white wall,close up portrait of a smiling middle aged woman without white wall,lexical/semantic,0.929199,0.070557,1
3,000000018,jpg,man sitting on floor beside a pool using laptop,"sitting,tablet computer,table,technology,grass,drinking,swimming pool,leisure,furniture,gadget,electronic device,vacation,laptop,stock photography,computer",man sitting on floor without a pool using laptop,https://media.gettyimages.com/photos/man-sitting-on-floor-beside-a-pool-using-laptop-picture-id588490995?s=612x612,man,"[{'object': 'floor', 'predicate': 'on'}, {'object': 'pool', 'predicate': 'beside a'}, {'object': 'laptop', 'predicate': 'using'}]",None,True,3,000000018.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000018.jpg,man sitting on floor beside a pool using laptop,man sitting on floor without a pool using laptop,lexical/semantic,0.930664,0.069519,1
4,000000007,jpg,a beautiful day with some buildings and plants .,"residential area,property,home,building,house,real estate,neighbourhood,town,architecture,estate,suburb,driveway,tree,road surface,road",a beautiful day without plants,https://d1tq208oegmb9e.cloudfront.net/site_photos_image/dbx%3A/urban+project/orange+county/fullerton/chapman+villas/Photos/3.jpg,day,"[{'object': 'buildings', 'predicate': 'with some'}, {'object': 'plants', 'predicate': 'with some'}]",None,True,2,000000007.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000007.jpg,a beautiful day with some buildings and plants,a beautiful day without plants,lexical/semantic,0.938965,0.060974,1
7,000000001,jpg,item : drawing of a figure surrounded by person,"drawing,modern art,line,visual arts,art,sketch,artwork,photographic paper,painting,illustration,black-and-white",item : drawing without a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd48780900641ded7ab53d74fe86fe--figure-painting-figure-drawing.jpg,drawing,"[{'object': 'figure', 'predicate': 'of a'}, {'object': 'person', 'predicate': 'surrounded by'}]",None,True,2,000000001.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000001.jpg,item drawing of a figure surrounded by person,item drawing without a figure surrounded by person,lexical/semantic,0.150024,0.850098,0


# Load Negclip CC12M Negfull ViT-B/32 LR-1e-8

In [31]:
checkpoint_path = "/workspace/Models/negclip/negclip_cc12m_negfull_vit_b_32_lr1e-8_clw_0_99_mlw_0_01.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "ViT-B/32"

model, preprocess = load_negclip_checkpoint(model_name, checkpoint_path, device)

In [32]:
results_df, score = evaluate_negation_sensitivity_clip(syntactic_df, model, preprocess, device)
print(f"Negation Sensitivity Score:, {score:.4f}")
results_df.head()

Evaluating with CLIP: 100%|██████████| 150735/150735 [1:41:24<00:00, 24.77it/s]  


Negation Sensitivity Score:, 0.7240


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
0,000000003,jpg,actor attends the season premiere,"musician,premiere,event,singer,suit,performance","actor, not attending the season premiere",https://media.gettyimages.com/photos/aidan-gillen-attends-the-season-7-premiere-of-hbos-game-of-thrones-at-picture-id817717986?s=612x612,actor,"[{'object': 'season premiere', 'predicate': 'attends the'}]",None,True,1,000000003.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000003.jpg,actor attends the season premiere,actor not attending the season premiere,syntactic,0.893066,0.106689,1
5,000000000,jpg,christmas tree on a black background .,"christmas tree,christmas decoration,font,text,graphic design,illustration,interior design,tree,christmas eve,ornament,fir,plant,pine,pine family,graphics","christmas tree, not on a black background",https://thumb1.shutterstock.com/display_pic_with_logo/261388/223876810/stock-vector-christmas-tree-on-a-black-background-vector-223876810.jpg,tree,"[{'object': 'background', 'predicate': 'on a'}]",None,True,1,000000000.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000000.jpg,christmas tree on a black background,christmas tree not on a black background,syntactic,0.730957,0.269043,1
6,000000011,jpg,festive banner with flags and an inscription .,"logo,flag,illustration,red,text,font,emblem,graphic design,graphics,label,symbol,art","festive banner with an inscription, but not with flags",https://thumb1.shutterstock.com/display_pic_with_logo/161878175/475029928/stock-vector-vector-festive-banner-with-flags-of-the-vietnam-and-an-inscription-socialist-republic-of-vietnam-475029928.jpg,banner,"[{'object': 'flags', 'predicate': 'with'}, {'object': 'inscription', 'predicate': 'with'}]",None,True,2,000000011.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000011.jpg,festive banner with flags and an inscription,festive banner with an inscription but not with flags,syntactic,0.699219,0.300781,1
9,000000024,jpg,actor attends the opening night after party,"premiere,carpet,red carpet,bow tie,hat,singer,fashion accessory,performance,tuxedo,formal wear,suit,musician",actor not attending the opening night after party,https://media.gettyimages.com/photos/actor-henry-kelemen-attends-the-our-new-girl-opening-night-after-at-picture-id450417500?s=612x612,actor,"[{'object': 'opening night after party', 'predicate': 'attends the'}]",None,True,1,000000024.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000024.jpg,actor attends the opening night after party,actor not attending the opening night after party,syntactic,0.708984,0.291016,1
13,000000029,jpg,a set of theatrical masks with different emotions,"white,face,mask,black-and-white,head,font,masque,headgear,costume,illustration,monochrome photography,pattern,smile,comedy","a set of theatrical masks, not with different emotions",https://thumb1.shutterstock.com/display_pic_with_logo/4089682/680796190/stock-vector-a-set-of-six-theatrical-masks-with-different-emotions-680796190.jpg,set,"[{'object': 'masks', 'predicate': 'of'}, {'object': 'emotions', 'predicate': 'with different'}]",None,True,2,000000029.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000029.jpg,a set of theatrical masks with different emotions,a set of theatrical masks not with different emotions,syntactic,0.569824,0.430176,1


In [33]:
results_df, score = evaluate_negation_sensitivity_clip(lexical_df, model, preprocess, device)
print(f"\nNegation Sensitivity Score: {score:.4f}")
results_df.head()

Evaluating with CLIP: 100%|██████████| 77511/77511 [55:30<00:00, 23.27it/s]   



Negation Sensitivity Score: 0.7347


,image_number,file_extension,true_caption,labels,negated_caption,url,subject,object_predicate_pairs,predicate,negate_word_present,num_ops,image_name,image_path,true_caption_clean,negated_caption_clean,negation_bucket,similarity_true,similarity_neg,true_greater_than_negated
1,000000005,jpg,a woman walks her dog on the beach .,"water,beach,sea,shore,ocean,canidae,dog,sky,wave,coast,mudflat,dog walking,human,sand,photography",a woman walks on the beach without her dog,https://media.gettyimages.com/photos/woman-walks-her-dog-on-the-beach-on-october-21-2014-in-saltcoats-picture-id457587968,woman,"[{'object': 'dog', 'predicate': 'walks her on the beach'}]",None,True,1,000000005.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000005.jpg,a woman walks her dog on the beach,a woman walks on the beach without her dog,lexical/semantic,0.672363,0.327637,1
2,000000009,jpg,close up portrait of a smiling middle aged woman sitting against white wall,"hair,sitting,facial expression,nose,arm,cheek,smile,chin,lip,hand,close-up,brown hair,finger,wool,fur",close up portrait of a smiling middle aged woman without white wall,http://l7.alamy.com/zooms/29164f933d7340be90af1ab4c91f3644/close-up-portrait-of-a-smiling-middle-aged-woman-sitting-against-white-hxeygn.jpg,portrait,"[{'object': 'woman', 'predicate': 'of a'}, {'object': 'wall', 'predicate': 'against white'}, {'object': 'age', 'predicate': 'middle'}, {'object': 'smile', 'predicate': 'smiling'}]",None,True,4,000000009.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000009.jpg,close up portrait of a smiling middle aged woman sitting against white wall,close up portrait of a smiling middle aged woman without white wall,lexical/semantic,0.928223,0.071594,1
3,000000018,jpg,man sitting on floor beside a pool using laptop,"sitting,tablet computer,table,technology,grass,drinking,swimming pool,leisure,furniture,gadget,electronic device,vacation,laptop,stock photography,computer",man sitting on floor without a pool using laptop,https://media.gettyimages.com/photos/man-sitting-on-floor-beside-a-pool-using-laptop-picture-id588490995?s=612x612,man,"[{'object': 'floor', 'predicate': 'on'}, {'object': 'pool', 'predicate': 'beside a'}, {'object': 'laptop', 'predicate': 'using'}]",None,True,3,000000018.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000018.jpg,man sitting on floor beside a pool using laptop,man sitting on floor without a pool using laptop,lexical/semantic,0.961426,0.038452,1
4,000000007,jpg,a beautiful day with some buildings and plants .,"residential area,property,home,building,house,real estate,neighbourhood,town,architecture,estate,suburb,driveway,tree,road surface,road",a beautiful day without plants,https://d1tq208oegmb9e.cloudfront.net/site_photos_image/dbx%3A/urban+project/orange+county/fullerton/chapman+villas/Photos/3.jpg,day,"[{'object': 'buildings', 'predicate': 'with some'}, {'object': 'plants', 'predicate': 'with some'}]",None,True,2,000000007.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000007.jpg,a beautiful day with some buildings and plants,a beautiful day without plants,lexical/semantic,0.964844,0.035156,1
7,000000001,jpg,item : drawing of a figure surrounded by person,"drawing,modern art,line,visual arts,art,sketch,artwork,photographic paper,painting,illustration,black-and-white",item : drawing without a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd48780900641ded7ab53d74fe86fe--figure-painting-figure-drawing.jpg,drawing,"[{'object': 'figure', 'predicate': 'of a'}, {'object': 'person', 'predicate': 'surrounded by'}]",None,True,2,000000001.jpg,/workspace/Dataset/conclip/ccneg_images/cc3m_subset_images_extracted_final/000000001.jpg,item drawing of a figure surrounded by person,item drawing without a figure surrounded by person,lexical/semantic,0.090088,0.909668,0
